# How to run the notebook

I developedd this on a V100.

Run the first cell that install the dependancies, then restart the runtime with runtime > restart runtime. This is necessary on colab because of the accellerate package.

After it restarted, run all cells excluding the one you already ran.

The results object should be logged at the end.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install transformers

!pip install torch

!pip install langchain
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install accelerate

!pip install einops

!pip install tqdm

In [1]:
#@title Medical record
medical_text = """
Medical Record
Pa#ent Name: John Smith
DOB: 06/16/1982
MRN: 456789123
Sex: Male
Chief Complaint
hemorrhoids, new patient
Pa/ent’s Care Team
Primary Care Provider: Name: Care Provider MD Address: 123 Main Street, Cocoa, FL 12345 Phone: (123) 456-
7899 Fax: (123) 456-7899 NPI: 1234567899
Sports Medicine: Name: Sports Medicine Address: 123 Main Street, Cocoa, FL 12345 Phone: (123) 456-7899
Fax: (123) 456-7899 NPI: 1234567899
Patient’s Pharmacies
Pharmacy: Name: Pharmacy Address: 123 Main Street, Cocoa, FL 12345 Phone: (123) 456-7899 Fax: (123) 456-
7899
Vitals
Ht: 6 P 2 in
Wt: 165 lbs
BMI: 21.2
Pulse: 96 bpm
RR: 16
02Sat: 87%
Allergies
• Allergies not reviewed (last reviewed 11/28/2022)
• NKDA
Medica/ons
• Clenpiq 10 rng-3.5 gram-12 gram/160 ml oral solution Take 320 ml by oral route as directed
• diclofenac 1 % topical gel BMI: 21.2 02Sat: 87% 02/10/23 prescribad 11 /28/22 filled · APPLY 2 GRAMS
TO THE AFFECTED AREA(S) TOPICALLY FOUR TIMES A DAY
• meloxicarn 15 mg tablet TAKE ONE TABLET BY MOUTH ONE TIME DAILY
Problems
Reviewed Problems · No known problems
Family History
Family History not reviewed (last reviewed 11/28/2022)
Mother
- Arthritis
- Hypertensive disorder
- Hypercholesterolemia
Father
- Hypertensive disorder
- Congestive heart failure
- Dlabatas mallitus

- Chronic obstructive lung disease
- Arthritis
- Heart disease
Brother
- Congestive heart failure
- Asthma
Social History
Social History not reviewed (last reviewed 11/28/2022)
Ac#vi#es of Dally Living
Are you deaf or do you have serious difficulty hearing?: No
Are you able to walk?: Yes: walks without restric@ons
Do you have transporta@on difficul@es?: No
Diet and Exercise
What type of diet are you following?: Regular
What Is your exercise level?: None
Educa#on and Occupa#on
What is the highest grade or level of school you have completed or the highest degree you have received?:
High school graduate
Are you currently employed?: Yes
What Is your occupation?: Lineman
Advance Directive
Do you have a medical power of attorney?: No
Is blood transfusion acceptable in an emergency?: Yes
Substance Use
Do you or have you eversmoked tobacco?: Currant every day smoker
How much tobacco do you chew?; none
At what age did you start smoking tobacco?: 20
How much tobacco do you smoke?: 1 pack per day
How many years have you smoked tobacco?: 20
What is your level of alcohol consump@on?: None
How many years have you consumed alcohol?: 20
What is your level of caffeine consump@on?: Occasional
Marriage and Sexuality
What is yourrela@onship status?: Married
Are you sexually ac@ve?: Yes
Do you use protec@on during sex?: No
How many children do you have?: 3
Home and Environment
Do you have any pets?: Yes
Are you passively exposed to smoke?: Yes
Lifestyle
Do you feel stressed (tense, restless, nervous, or anxious, or unable to sleep at night)?: Not at all
Do you use your seat belt or car seat rou@nely?: No
PMG Social History
Do you have a supportsystem?: Yes
Domes@c violence history: No

Contracep@on used: Vasectomy
Other tobacco products: None
Living will: No
Educa@on specifics: Public school
Home health; No
Received rabies vaccine in 2002 - travelled to Namibia
Gender Iden#ty and LGBTQ
Iden@ty GenderIden@ty: Choose not to disclose
First name used: John
Sexual orienta@on: Choose not to disclose
Surgical History
Surgical History not reviewed (last reviewed 11/28/2022)
Past Medical History
Patient diagnosed with haemorrhoids in 2009, treated with hydrocortisone cream, and fully resolved

H P I
Pa@ent is a 41 y/o male with history of lateral epicondyli@s who isseen for the first @me at this prac@ce.
Pa@ent never had a colonoscopy. Pa@ent is complaining ofsymptoma@c haemorrhoidstwo months ago, one of
which had ruptured and bled last year. Pa@ent hasfamily history of colon cancer (grandfather on his eigh@es).
Pa@ent denies blood in stool.
ROS
ROS as noted in the HPJ
Physical Exam
Cons#tu#onal: General Appearance: healthy-appearing, well-nourished, and well-developed. Level of Distress:
NAO. Ambula@on: ambula@ng normally.
Psychiatric: Insight: good Judgement. Mental Status: ac@ve and alert. Orienta@on: to @me, place, and person.
Memory: recent memory normal.
Abdomen: Bowel Sounds: normal. Inspec@on and Palpa@on: no tenderness, guarding, masses, rebound
tenderness, or CVA tenderness and soP and non-distended. Liver: non-tender and no hepatomegaly. Spleen:
non-tender and no splenomegaly. Hernia: none palpable.
Assessment/ Plan
1. Haemorrhoids
Pa@ent is asymptoma@c right know. I advised to seek evalua@on for banding procedure aPer colonoscopy
(depending on evalua@on done)
K64.9: Unspecified haemorrhoids.
2. Screening for malignant neoplasm of colon
Pa@ent will have colonoscopy with average risk. Consentsigned. Risks associated with this procedure such as
bleeding, Infec@on and perfora@on were discussed with the pa@ent.
Z12.11: Encounter for screening for malignant neoplasm of colon
• Clenpiq 10 mg-3.5 gram-12 gram/160 ml oral solu#on - Take 320 ml by oral route as directed Qty:
(320) ml Refills: 0
3. Family history of cancer of colon-

Pa@ent has family history of colon cancer (grandfather on his eigh@es) Z80.0: Family history of malignant
neoplasm of diges@ve organs
Haematochezia
4. Pa@ent has BRBPR last year
K92.1: Melena
"""

In [2]:
#@title Data model
import json
from jsonschema import validate

question_schema = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "version": "1.0.0",
  "properties": {
    "question": {"type": "string"},
    "answer": {"type": "string"},
    "justification": {"type": "string"},
    "confidence_score": {"type": "integer"}
  },
  "required": ["question", "answer", "justification", "confidence_score"],
}

schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "version": "1.0.0",
    "properties": {
        "general_info": {
            "type": "object",
            "properties": {
                "chief_complaint": {"type": "string"},
                "allergies": {"type": "string"},
                "preexisting_medications": {
                    "type": "string",
                },
                "meta": {
                    "type": "object",
                    "properties": {
                        "risk_profile": {"type": "string"},
                        "treatment_assessment": {"type": "string"}
                    },
                    "required": ["risk_profile", "treatment_assessment"]
                }
            },
            "required": ["chief_complaint", "treatment_plan", "allergies", "preexisting_medications", "meta"]
        },
        "questions": {
            "type": "array",
            "items": question_schema
        },
    },
    "required": ["general_info", "questions"]
}

# Initialize objects based on schema
def initialize_object_from_schema(schema):
    if "type" not in schema:
        return None

    if schema["type"] == "object":
        obj = {}
        for prop, prop_schema in schema.get("properties", {}).items():
            obj[prop] = initialize_object_from_schema(prop_schema)
        return obj

    elif schema["type"] == "array":
        return []

    elif schema["type"] == "string":
        return ""

    elif schema["type"] == "integer":
        return 0

    elif schema["type"] == "number":
        return 0.0

    elif schema["type"] == "boolean":
        return False

    elif schema["type"] == "null":
        return None

    return None



In [3]:
#@title LLM utilities
import os
import torch
from langchain import HuggingFacePipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

def load_llm_model(model_id="mistralai/Mistral-7B-Instruct-v0.1", use_quantization=True):
    print(model_id)
    quantization_config = None

    if use_quantization:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        offload_folder="/tmp",
        quantization_config=quantization_config,
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

    return model, tokenizer

def setup_llm_pipeline(model, tokenizer):
    pp = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=3000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    return HuggingFacePipeline(pipeline=pp)

def unload_llm_model(model):
    del model
    torch.cuda.empty_cache()


In [4]:
#@title Constants
#### Prompt
short_answer_template = """[INST]Please read the following context and answer the question at the end. Start with `Answer:` followed by the answer. The answer should be very short.
Example Output:
Answer: Glaucoma.
[/INST]

Context: {context}
Question: {question}
"""

two_part_template = """[INST]Please read the following context and answer the question at the end. Start with `Answer:` followed by 'Yes' or 'No,' and then write `Justification:` followed by your reasoning.
Example Output:
Answer: Yes
Justification: The evidence in the context strongly supports a 'Yes' answer.
[/INST]

Context: {context}
Question: {question}
"""

assessement_template = """[INST]Please read the following doctor treatment plan and classify it. The categories are "mild", "average", "aggressive".
Example of a good answer: "Answer: mild".[/INST]
Context: {context}
"""

base_questions = {
  "chief_complaint": "What is the patient main complaint?",
  "treatment_plan": "what treatment plan is the doctor suggesting?",
  "allergies":"Does the patient have allergies and if so, which ones?",
  "preexisting_medications":"Is the patient on medications?"
}

specific_questions = [
  "Does the patient have a family history of colon cancer in their first-degree relatives?",
  "Has the patient experienced minimal bright red blood per rectum?",
  "Has the patient had significant loss of blood?"
  "Does the patient have a history of skin problems?",
  "Has the patient used hydrocortisone cream for the haemorrhoids that they are recently experiencing?",
  "Were any high risk traits found on the patient’s genetic test?",
  "Has the patient had a colonoscopy in the last 5 years?",
  "Has the patient had any recent foreign travel?",
  "How long has the patient been known to healthcare services?"
]


In [5]:
#@title Analyse Document
import re
from tqdm import tqdm

def extract_answer_and_justification(text):
    answer_pattern = r"Answer: (Yes|No)"
    justification_pattern = r"Justification: (.+)"

    answer_match = re.search(answer_pattern, text)
    justification_match = re.search(justification_pattern, text)

    answer = "Default Answer"  # Default value for answer
    justification = "Default Justification"  # Default value for justification

    if answer_match:
        answer = answer_match.group(1)

    if justification_match:
        justification = justification_match.group(1)

    return answer, justification


from langchain import PromptTemplate, LLMChain

def analyse_document(llm, initial_object, base_questions, specific_questions, context_p):

    print(initial_object)

    for key, question in tqdm(base_questions.items(), desc="Extracting general info.."):

      prompt = PromptTemplate(template=short_answer_template, input_variables=["question","context"])
      llm_chain = LLMChain(prompt=prompt, llm=llm)

      response = llm_chain.run({"question":question,"context":context_p})
      initial_object["general_info"][key] = response.lstrip('Answer:')

      if key == "treatment_plan":
        prompt = PromptTemplate(template=assessement_template, input_variables=["context"])
        llm_chain = LLMChain(prompt=prompt, llm=llm)
        response = llm_chain.run({"context": response})

        initial_object["general_info"]["meta"]["treatment_assessment"] = response.lstrip('Answer:')



    for question in tqdm(specific_questions, desc="Answering questions.."):
      question_item = initialize_object_from_schema(question_schema)

      prompt = PromptTemplate(template=two_part_template, input_variables=["question","context"])
      llm_chain = LLMChain(prompt=prompt, llm=llm)

      response_text = llm_chain.run({"question":question,"context":context_p})

      answer, justification = extract_answer_and_justification(response_text)

      question_item["answer"] = answer.lstrip('Answer:')
      question_item["justification"] = justification.lstrip('Answer:')
      question_item["question"] = question

      initial_object["questions"].append(question_item)



    return initial_object

In [6]:
#@title Confidence scoring
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


from functools import partial


def load_model_and_tokenizer(questions_list):
    global model, tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    return questions_list

def unload_model_and_tokenizer(questions_list):
    global model, tokenizer
    del model
    del tokenizer
    model = None
    tokenizer = None
    torch.cuda.empty_cache()
    return questions_list

def calculate_and_normalize_perplexity(questions_list):
    if model is None or tokenizer is None:
        print("Model and tokenizer are not loaded. Please load them first.")
        return

    for item in questions_list:
      if not isinstance(item, dict):
        print(f"Skipping item {item} because it's not a dictionary")
        continue

      if "question" not in item or "justification" not in item:
        print(f"Skipping item {item} because it lacks required keys")
        continue

      question = item["question"]
      justification = item["justification"]

      text = question + " " + justification
      input_ids = tokenizer.encode(text, return_tensors="pt")

      with torch.no_grad():
          output = model(input_ids, labels=input_ids)
          log_likelihood = output.loss

      perplexity = torch.exp(log_likelihood).item()

      # Normalize perplexity
      max_perplexity = 500
      perplexity = min(max(perplexity, 1), max_perplexity)
      normalized_score = 1 + 9 * ((perplexity - 1) / (max_perplexity - 1))
      inverted_score = 11 - normalized_score

        # Update confidence_score
      item["confidence_score"] = int(round(inverted_score))

    return questions_list


In [7]:
#@title Utils
# Function to convert data to JSON and validate
def convert_to_json_and_validate(data, schema):
    json_data = json.dumps(data)

    try:
        validate(instance=data, schema=schema)
        print("JSON data is valid.")
    except Exception as e:
        print(f"JSON data is invalid: {e}")

    return json_data

def categorize_risk(data):
  total_questions = len(data.get('questions', []))
  one_third = total_questions / 3
  two_thirds = 2 * one_third

  count = 0
  for question in data.get('questions', []):
      answer = question.get('answer', '').strip().lower()
      if answer == 'yes':
          count += 1

  if count < one_third:
      return 'Low'
  elif count < two_thirds:
      return 'Average'
  else:
      return 'High'

In [8]:
#@title Main
from typing import List, Dict, Any

def main_pipeline(
    medical_text: str,
    schema: Dict[str, Any],
    base_questions: List[Dict[str, Any]],
    specific_questions: List[Dict[str, Any]]
) -> Dict[str, Any]:
    try:
        use_quantization = not os.environ.get("DISABLE_QUANTIZATION", False)

        model_id_1 = "mistralai/Mistral-7B-Instruct-v0.1"

        # Initialize DTO from schema
        dto = initialize_object_from_schema(schema)

        # Load first LLM model and analyze document
        model, tokenizer = load_llm_model(model_id_1, use_quantization=use_quantization)
        llm = setup_llm_pipeline(model, tokenizer)
        dto = analyse_document(llm, dto, base_questions, specific_questions, medical_text)

        confidence = calculate_and_normalize_perplexity(load_model_and_tokenizer(dto["questions"]))
        dto["questions"] = confidence

        unload_llm_model(model)


        dto["general_info"]["meta"]["risk_profile"] = categorize_risk(dto)

        return convert_to_json_and_validate(dto, schema)

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}


# Run the pipeline
result = main_pipeline(medical_text, schema, base_questions, specific_questions)

json.loads(result)

mistralai/Mistral-7B-Instruct-v0.1

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'general_info': {'chief_complaint': '', 'allergies': '', 'preexisting_medications': '', 'meta': {'risk_profile': '', 'treatment_assessment': ''}}, 'questions': []}


Answering questions..:  62%|██████▎   | 5/8 [00:21<00:12,  4.15s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Answering questions..: 100%|██████████| 8/8 [00:41<00:00,  5.17s/it]


JSON data is valid.


{'general_info': {'chief_complaint': ' Hemorrhoids.',
  'allergies': '\nAnswer: None mentioned in the medical record. However, it should be noted that allergies are not reviewed in the context provided.',
  'preexisting_medications': ' Yes',
  'meta': {'risk_profile': 'Average',
   'treatment_assessment': "\nAnswer: aggressive\n\nThe treatment plan for hemorrhoids is banding procedure through colonoscopy, which is an aggressive approach to treat hemorrhoids. This procedure is typically reserved for severe cases of hemorrhoids and is recommended for patients who are experiencing significant bleeding or discomfort. The patient's family history of colon cancer and risk for developing malignant neoplasm of colon make them a higher risk for developing hemorrhoids and require further investigation. The patient should seek an evaluation for the banding procedure as soon as possible to prevent any potential complications that may arise from the bleeding."},
  'treatment_plan': ' Treatment plan